In [1]:
import pandas as pd

In [2]:
base_path = '/Users/rustomichhaporia/GitHub/KeyClass/team_17/data/'

diagnoses_icd = pd.read_csv(base_path + 'DIAGNOSES_ICD.csv', usecols=['HADM_ID', 'ICD9_CODE']).dropna()
note_events = pd.read_csv(base_path + 'NOTEEVENTS.csv', usecols=['HADM_ID', 'CATEGORY', 'TEXT'])
note_events = note_events[note_events['CATEGORY'] == 'Discharge summary']

output = pd.merge(diagnoses_icd, note_events, on='HADM_ID', how='inner').drop_duplicates()
output = output.sample(100000)
output.to_csv(base_path + 'mimic_subset.csv', index=False)

In [3]:
def process_data(raw_data):
    # implement this function to process the data as you need
    def get_high_level_code(raw_code):
        high_level_codes = {
            0: (0, 139),
            1: (140, 239),
            2: (240, 279),
            3: (280, 289),
            4: (290, 319),
            5: (320, 389),
            6: (390, 459),
            7: (460, 519),
            8: (520, 579),
            9: (580, 629),
            10: (630, 679),
            11: (680, 709),
            12: (710, 739),
            13: (740, 759),
            14: (760, 779),
            15: (780, 799),
            16: (800, 999),
            17: (1000, 2000),
            18: (2001, 3000),
        }

        high_level_code = 0
        if raw_code.startswith('E'):
                high_level_code = 1000
        elif raw_code.startswith('V'):
            high_level_code = 1000
        elif raw_code.startswith('U'):
            high_level_code = 2001
        high_level_code = int(raw_code[1:])

        for high_level, (bottom, top) in high_level_codes.items():
            if bottom <= high_level_code <= top:
                return high_level
    raw_data['ICD9_CODE'] = raw_data['ICD9_CODE'].apply(get_high_level_code)
    return raw_data.dropna()
            

In [21]:
processed_data = process_data(output)

AttributeError: 'float' object has no attribute 'startswith'

In [22]:
processed_data = processed_data.sample(1000)
processed_data.to_csv(base_path + 'mimic_subset_processed.csv', index=False)

In [23]:
processed_data['TEXT'] = processed_data['TEXT'].str.replace('\n', ' ').replace('\r', '')

In [24]:
from sklearn.model_selection import train_test_split

X = processed_data['TEXT']
y = processed_data['ICD9_CODE']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [26]:
base_path += 'mimic-iii/'
X_train.to_csv(base_path + 'train.txt', index=False, header=False)
X_test.to_csv(base_path + 'test.txt', index=False, header=False)
y_train.astype(int).to_csv(base_path + 'train_labels.txt', index=False, header=False)
y_test.astype(int).to_csv(base_path + 'test_labels.txt', index=False, header=False)